In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-108fya84
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-108fya84
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=5874dbbce7bb70dfd258202b85e35fb961a912628daf6bd523ec0d9e8948c476
  Stored in directory: /tmp/pip-ephem-wheel-cache-du4dbrwj/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [57]:
%%cuda --name SDL.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

#define THREADxBLOCK 128

using namespace std;

__global__ void setDegrees(GraphStruct *str, uint k, uint weight){
    uint n = str->nodeSize;

    for (int idx = threadIdx.x + blockIdx.x * blockDim.x; idx < n; idx += blockDim.x*gridDim.x) {
        uint offset = str->cumDegs[idx];
        uint originalDeg = str->cumDegs[idx + 1] - str->cumDegs[idx];
        uint inducedDeg = 0;

        for (uint i = 0; i < originalDeg; i++){
            uint neighID = str->neighs[offset + i];
            if(str->weights[neighID] == -1) inducedDeg += 1;
        }

        if (inducedDeg <= k && str->weights[idx] == -1) str->weights[idx] = weight;
    }
}

void initDegrees(GraphStruct *str){
    uint degree = 1;
    uint weigth = 1;
    uint n = str->nodeSize;

    dim3 threads ( THREADxBLOCK);
    dim3 blocks ((n + threads.x - 1) / threads.x, 1, 1 );

    bool flag = true;

    while(flag){
        for(int i = 0; i < n; i++){
            setDegrees <<< 1, 1 >>> (str, degree, weigth);
            cudaDeviceSynchronize();

            flag = false;

            for(int j = 0; j < n; j++){
                if(str->weights[j] == -1) flag = true;
		        }
            for(int j = 0; j < n; j++){
                if(str->weights[j] == weigth){
                    weigth += 1;
                    break;
                }
		        }

            if(!flag) break;

        }
        degree += 1;
    }
}

__global__ void findCandidates (Coloring* col, GraphStruct *str, uint* degrees, uint* weigths, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
    for (uint j = 0; j < deg; j++) {
	    uint neighID = str->neighs[offset + j];
		if (col->coloring[neighID] == 0 &&
				((degrees[idx] < degrees[neighID]) ||
				((degrees[idx] = degrees[neighID]) && (weigths[idx] < weigths[neighID])))) {
			candidate = false;
		}
	}

    if(candidate){
        candidateNodes[idx] = true;
    }
}

__global__ void colorer(Coloring * col, GraphStruct *str){
	int n = str->nodeSize;

	for (int i = threadIdx.x+blockIdx.x*blockDim.x; i < n; i += blockDim.x*gridDim.x) {
		bool flag = true; // vera sse il nodo ha peso locale massimo

		// ignora i nodi già colorati
		if ((col->coloring[i] != -1)) continue;

		int iWeight = str->weights[i];

		// guarda i pesi del vicinato
		uint offset = str->cumDegs[i];
		uint deg = str->cumDegs[i + 1] - str->cumDegs[i];

		for (uint j = 0; j < deg; j++) {
			uint neighID = str->neighs[offset + j];
			int jColor = col->coloring[neighID];

			if ((jColor != -1) || (i == neighID)) continue;

			int jWeight = str->weights[neighID];
			uint neighDeg = str->cumDegs[neighID + 1] - str->cumDegs[neighID];
			if ((deg < neighDeg) || ((deg == neighDeg) && (iWeight < jWeight))) flag = false;
		}

		// colora solo se sei il nodo di peso massimo
		if (flag){
			bool* forbidden;
			cudaMalloc((void**) &forbidden, n * sizeof(bool));
			memset(forbidden, false, n);

			for (uint j = 0; j < deg; j++) {
				uint neighID = str->neighs[offset + j];
				if(col->coloring[neighID] != -1) forbidden[col->coloring[neighID]] = true;

			}

			for(uint j = 0; j < n; j++){
				if(forbidden[j] == false){
					col->coloring[i] = j;
					break;
				}
			}
		free(forbidden);
		}
	}
}

void h_swap(int* array, int idx_a, int idx_b){
    int tmp = array[idx_a];
    array[idx_a] = array[idx_b];
    array[idx_b] = tmp;

    return;
}

void FYshuffle(int * weights, uint n){
    for(int i = 0; i < n; i++){
        int swapIdx = (rand() % (n - i)) + i;
        h_swap(weights, i, swapIdx);
    }
}

Coloring* graphColoring(GraphStruct *str) {
	int n = str->nodeSize;
	Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	col->uncoloredNodes = true;

    CHECK(cudaMallocManaged(&(col->coloring), n * sizeof(int)));
	memset(col->coloring, -1 ,n * sizeof(int));
	// allocate space on the GPU for the random states

	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

    initDegrees(str);
	for (int i = 0; i < n; i++){
        printf("%d ", str->weights[i]);
    }
    printf("\n");

  //print_d <<< 1, 1 >>> (str, true);

	bool flag = false;
	for(int c = 0; c < n; c++){
		col->numOfColors = c;
		colorer<<<blocks, threads>>>(col, str);
		cudaDeviceSynchronize();

		for(int i=0; i<n; i++){
				if(col->coloring[i] == -1){
					flag = true;
				}
		}

		if(!flag) break;
		else flag = false;

	}

    return col;
}

'File written in /content/src/SDL.cu'

In [55]:
%%cuda --name test_SDL.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"


int main(void) {
	unsigned int n = 100;		 // number of nodes for random graphs
	float prob = 1;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

	cudaEventRecord(stop);
  cudaEventSynchronize(stop);

	//Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	uint maxColor = col->coloring[0];
 	printf("Coloratura trovata: ");
	for(int i = 0; i < str->nodeSize; i++){
			if(maxColor < col->coloring[i]) maxColor = col->coloring[i];
			printf("%d ", col->coloring[i]);
	}
	printf("\nColore massimo: %d", maxColor+1);

	return EXIT_SUCCESS;
}

'File written in /content/src/test_SDL.cu'

In [58]:
!nvcc -dc src/test_SDL.cu /content/src/SDL.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc test_SDL.o SDL.o graph.o graph_d.o -o test_SDL
!./test_SDL

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
5323.665527 ms
Coloratura trovata: 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 
Colore massimo: 3